# Financial Analysis with LLMs

This notebook demonstrates how to leverage Large Language Models (LLMs) and financial data APIs to perform financial analysis. We'll be using:

1. OpenAI's GPT model for natural language processing and analysis
2. Yahoo Finance API for retrieving real-time financial data
3. LangChain for creating a seamless integration between the LLM and financial data

Throughout this notebook, we'll explore how to:

- Set up the necessary environment and dependencies
- Initialize and use an LLM for financial queries
- Connect the LLM to real-time financial data using Yahoo Finance
- Create an agent that can answer complex financial questions by combining LLM capabilities with live market data

Let's dive in and explore the intersection of AI and finance!


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%pip install openai  langchain langchain-openai yfinance agno

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
llm.invoke("What is the stock price of Apple?")

AIMessage(content="I'm unable to provide real-time stock prices or any live data, including the current stock price of Apple. To find the most up-to-date stock price, you can check financial news websites, stock market apps, or a financial service like Bloomberg or Yahoo Finance.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 15, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-914f0f6c-2b60-4c54-8265-7a1ed893057c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 53, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Connecting LLMs to Financial Data

Using Yahoo Finance API

In [ ]:
import yfinance as yf
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(temperature=0)

# Define the function to get stock price
def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return f"The current stock price of {symbol} is ${todays_data['Close'].iloc[-1]:.2f}"

# Create a tool for the agent to use
tools = [
    Tool(
        name="StockPrice",
        func=get_stock_price,
        description="Useful for getting the stock price of a company. The input should be the stock symbol of the company."
    )
]

In [ ]:
# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

# Example usage
query = "What's the current stock price of Apple?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...
I should use the StockPrice tool to get the current stock price of Apple.
Action: StockPrice
Action Input: AAPL
Observation: The current stock price of AAPL is $239.47
Thought:I need to provide the answer to the original question.
Final Answer: The current stock price of Apple is $239.47.

> Finished chain.
{'input': "What's the current stock price of Apple?", 'output': 'The current stock price of Apple is $239.47.'}


In [ ]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def get_stock_price(symbol):
    """Use this tool to get the stock price of a company. The input should be the stock symbol of the company."""
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return f"The current stock price of {symbol} is ${todays_data['Close'].iloc[-1]:.2f}"

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain import hub

tools = [get_stock_price]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model = "gpt-4o-mini")

finance_agent = create_tool_calling_agent(llm, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [ ]:
from langchain_core.messages import HumanMessage

response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the stock price of Apple?")]})
response




> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'AAPL'}`


The current stock price of AAPL is $239.21The current stock price of Apple (AAPL) is $239.21.

> Finished chain.


{'messages': [HumanMessage(content='What is the stock price of Apple?', additional_kwargs={}, response_metadata={})],
 'output': 'The current stock price of Apple (AAPL) is $239.21.'}

In [ ]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the last dividend date of Apple?")]})
response

Adding Stock Market Analysis Tools

In [ ]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""

    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)

    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()

    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    inst_holders = ticker_obj.get_institutional_holders()

    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)

    curr_year = date.today().year

    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]

    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()

    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)

    return ticker_obj.get_news()

In [ ]:
tools = [
    company_information,
    last_dividend_and_earnings_date,
    stock_splits_history,
    summary_of_mutual_fund_holders,
    summary_of_institutional_holders,
    stock_grade_updrages_downgrades,
    stock_news,
    get_stock_price
]


finance_agent = create_tool_calling_agent(llm, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [ ]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the last dividend date of Apple?")]})
response



> Entering new AgentExecutor chain...

Invoking: `last_dividend_and_earnings_date` with `{'ticker': 'AAPL'}`


{'Dividend Date': datetime.date(2025, 2, 13), 'Ex-Dividend Date': datetime.date(2025, 2, 10), 'Earnings Date': [datetime.date(2025, 4, 30), datetime.date(2025, 5, 5)], 'Earnings High': 1.67, 'Earnings Low': 1.5, 'Earnings Average': 1.61721, 'Revenue High': 95903397000, 'Revenue Low': 89370000000, 'Revenue Average': 94063213870}The last dividend date for Apple (AAPL) is February 13, 2025. The ex-dividend date is February 10, 2025.

> Finished chain.


{'messages': [HumanMessage(content='What is the last dividend date of Apple?', additional_kwargs={}, response_metadata={})],
 'output': 'The last dividend date for Apple (AAPL) is February 13, 2025. The ex-dividend date is February 10, 2025.'}

In [ ]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the latest news about Apple?")]})
response



> Entering new AgentExecutor chain...

Invoking: `stock_news` with `{'ticker': 'AAPL'}`


[{'id': '4e4a6c3f-be05-4a45-b937-e33993468f01', 'content': {'id': '4e4a6c3f-be05-4a45-b937-e33993468f01', 'contentType': 'STORY', 'title': 'Big Tech antitrust fights face a new uncertainty: Trump', 'description': '', 'summary': "Alphabet’s Google is among the Silicon Valley giants hoping for a reprieve from the new antitrust enforcers in Donald Trump's new administration.", 'pubDate': '2025-03-07T09:00:11Z', 'displayTime': '2025-03-07T09:00:11Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-03/a6841950-fae3-11ef-a7c8-41e9dcf6b5e1', 'originalWidth': 3241, 'originalHeight': 2161, 'caption': '', 'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/ZLGpo3qKF.oUzPzQ_ACowQ--~B/aD0yMTYxO3c9MzI0MTthcHBpZD15dGFjaHlvbg--/https://s.yimg.com/os/creatr-uploaded-images/2025-03/a6841950-fae3-11ef-a7c8-41e9dcf6b

{'messages': [HumanMessage(content='What is the latest news about Apple?', additional_kwargs={}, response_metadata={})],
 'output': "Here are some of the latest news articles about Apple:\n\n1. **Title:** [Big Tech antitrust fights face a new uncertainty: Trump](https://finance.yahoo.com/news/big-tech-antitrust-fights-face-a-new-uncertainty-trump-090011456.html)\n   - **Summary:** Alphabet’s Google is among the Silicon Valley giants hoping for a reprieve from the new antitrust enforcers in Donald Trump's new administration.\n   - **Published On:** March 7, 2025\n   - **Source:** Yahoo Finance\n\n2. **Title:** [Apple Secures Local Content Certifications in Indonesia but Faces Additional Regulatory Hurdles](https://finance.yahoo.com/news/apple-secures-local-content-certifications-153755274.html)\n   - **Summary:** Apple secures local content certificates for 20 products in Indonesia, including the iPhone 16.\n   - **Published On:** March 7, 2025\n   - **Source:** GuruFocus.com\n\n3. **Ti

In [ ]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")]})
response



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'NVDA'}`


{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '(408) 486-2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation, a computing infrastructure company, provides graphics and compute and networking solutions in the United States, Singapore, Taiwan, China, Hong Kong, and internationally. The Compute & Networking segment comprises Data Center computing platforms and end-to-end networking platforms, including Quantum for InfiniBand and Spectrum for Ethernet; NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other embedded platforms; NVIDIA AI Enterprise an

{'messages': [HumanMessage(content='What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?', additional_kwargs={}, response_metadata={})],
 'output': 'For Nvidia (NVDA), the financial metrics are as follows:\n\n- **EBITDA**: $83,316,998,144\n- **Total Debt**: $10,270,000,128\n- **Total Revenue**: $130,497,003,520\n- **Debt-to-Equity Ratio**: 12.946\n\nIf you need further information, feel free to ask!'}

In [ ]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="Given the current market trends is it good to invest in Nvidia")]})
response

# Stock Maket Agnet with Agno

-  AI & Finance: A Perfect Match 🤝
-  In this notebook, we'll explore how AI can revolutionize investing by analyzing stock trends.




In [ ]:
%pip install agno duckduckgo-search yfinance openai matplotlib google-search-results openai

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

###Import AI Tools


In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### Create AI Agents :- Uses Yahoo Finance Tool
- Web Agent searches the internet, Finance Agent pulls stock data

In [ ]:
web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools()],
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=OpenAIChat(id="gpt-4o"),
    tools=[YFinanceTools(stock_price=True, company_info=True)],
    instructions=["Always use tables to display data"],
    markdown=True,
)

team = Agent(team=[web_agent, finance_agent], markdown=True)

### Analyze Tech Sector Trends


In [ ]:
query = """
Analyze the performance of the tech sector over the last 6 months using these stocks: AAPL, MSFT, GOOGL.
Include:
1. Average price change across these stocks
2. Key events or trends from the web affecting the sector
"""
response = team.print_response(query, stream=True)

Output()

### Screen Stocks by Risk


In [ ]:
query = """
I’m a moderate-risk investor. Suggest 3 stocks to consider for a 5-year investment.
Include:
1. Recent performance (6 months)
2. Why they fit moderate risk (e.g., stability, growth potential)
"""
response = team.print_response(query, stream=True)

Output()

### Define and Test Portfolio


In [ ]:
portfolio = """
Portfolio:
1. NVDA (NVIDIA): 40 shares
2. KO (Coca-Cola): 60 shares
3. SPY (S&P 500 ETF): 20 shares

Stress-test this portfolio:
1. How did these stocks perform in the last market downturn (e.g., early 2023)?
2. What’s the risk level of this mix?
3. Suggestions to make it safer for a 5-year moderate-risk plan?
"""
response = team.print_response(portfolio, stream=True)

Output()